In [1]:
import pandas as pd

In [3]:
df=pd.read_csv("heart_clean_final.csv")

In [46]:
df2=df

In [34]:
indep=df2.drop("target",axis=1)
dep=df2["target"]

In [9]:
def split_scalar(indep,dep):
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    X_train,X_test,Y_train,Y_test=train_test_split(indep,dep,test_size=0.2,random_state=0)
    sc=StandardScaler()
    X_train_sc=sc.fit_transform(X_train)
    X_test_sc=sc.transform(X_test)
    return X_train_sc,X_test_sc,Y_train,Y_test

In [14]:
def PCA_Classification(X_train_sc,X_test_sc,n):
    from sklearn.decomposition import PCA
    pca=PCA(n_components=n)
    X_train_pca=pca.fit_transform(X_train_sc)
    X_test_pca=pca.transform(X_test_sc)
    explained_variance=pca.explained_variance_ratio_
    return X_train_pca,X_test_pca,explained_variance

In [16]:
def cm_prediction(classifier,X_test_pca,Y_test):
    y_pred=classifier.predict(X_test_pca)
    from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
    cm=confusion_matrix(Y_test,y_pred)
    accuracy=accuracy_score(Y_test,y_pred)
    report=classification_report(Y_test,y_pred)
    return accuracy,report,cm

In [18]:
def logistic(X_train_pca,X_test_pca,Y_train,Y_test):
    from sklearn.linear_model import LogisticRegression
    classifier=LogisticRegression(random_state=0)
    classifier.fit(X_train_pca,Y_train)
    accuracy,report,cm=cm_prediction(classifier,X_test_pca,Y_test)
    return classifier,accuracy,report,cm

In [20]:
def svm_linear(X_train_pca,X_test_pca,Y_train,Y_test):
    from sklearn.svm import SVC
    classifier=SVC(kernel = 'linear', random_state = 0)
    classifier.fit(X_train_pca,Y_train)
    accuracy,report,cm=cm_prediction(classifier,X_test_pca,Y_test)
    return classifier,accuracy,report,cm

In [22]:
def svm_NL(X_train_pca,X_test_pca,Y_train,Y_test):
    from sklearn.svm import SVC
    classifier=SVC(kernel = 'rbf', random_state = 0)
    classifier.fit(X_train_pca,Y_train)
    accuracy,report,cm=cm_prediction(classifier,X_test_pca,Y_test)
    return classifier,accuracy,report,cm
    

In [24]:
def Navie(X_train_pca,X_test_pca,Y_train,Y_test):
    from sklearn.naive_bayes import GaussianNB
    classifier=GaussianNB()
    classifier.fit(X_train_pca,Y_train)
    accuracy,report,cm=cm_prediction(classifier,X_test_pca,Y_test)
    return classifier,accuracy,report,cm
    

In [26]:
def knn(X_train_pca,X_test_pca,Y_train,Y_test):
    from sklearn.neighbors import KNeighborsClassifier
    classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    classifier.fit(X_train_pca,Y_train)
    accuracy,report,cm=cm_prediction(classifier,X_test_pca,Y_test)
    return classifier,accuracy,report,cm
    

In [28]:
def Decision(X_train_pca,X_test_pca,Y_train,Y_test):
    from sklearn.tree import DecisionTreeClassifier
    classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
    classifier.fit(X_train_pca,Y_train)
    accuracy,report,cm=cm_prediction(classifier,X_test_pca,Y_test)
    return classifier,accuracy,report,cm

In [30]:
def random(X_train_pca,X_test_pca,Y_train,Y_test):
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(X_train_pca,Y_train)
    accuracy,report,cm=cm_prediction(classifier,X_test_pca,Y_test)
    return classifier,accuracy,report,cm

In [36]:
X_train_sc,X_test_sc,Y_train,Y_test=split_scalar(indep,dep)

In [38]:
acclog=[]
accsvml=[]
accsvmnl=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]
for n in range(1,6):
    X_train_pca,X_test_pca,explained_variance=PCA_Classification(X_train_sc,X_test_sc,n)
    classifier,accuracy,cm,report=logistic(X_train_pca,X_test_pca,Y_train,Y_test)
    acclog.append(accuracy)
    classifier,accuracy,cm,report=svm_linear(X_train_pca,X_test_pca,Y_train,Y_test)
    accsvml.append(accuracy)
    classifier,accuracy,cm,report=svm_NL(X_train_pca,X_test_pca,Y_train,Y_test)
    accsvmnl.append(accuracy)
    classifier,accuracy,cm,report=knn(X_train_pca,X_test_pca,Y_train,Y_test)
    accknn.append(accuracy)
    classifier,accuracy,cm,report=Navie(X_train_pca,X_test_pca,Y_train,Y_test)
    accnav.append(accuracy)
    classifier,accuracy,cm,report=Decision(X_train_pca,X_test_pca,Y_train,Y_test)
    accdes.append(accuracy)
    classifier,accuracy,cm,report=random(X_train_pca,X_test_pca,Y_train,Y_test)
    accrf.append(accuracy)

In [40]:
def pca_classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf):
    dataframe=pd.DataFrame(index=['PC-1','PC-2','PC-3','PC-4','PC-5'],columns=['Logistic','SVMl','SVMnl','KNN','Navie','Decision','Random'])
    for number,idex in enumerate(dataframe.index):
        dataframe.loc[idex,'Logistic']=acclog[number]
        dataframe.loc[idex,'SVMl']=accsvml[number]
        dataframe.loc[idex,'SVMnl']=accsvmnl[number]
        dataframe.loc[idex,'KNN']=accknn[number]
        dataframe.loc[idex,'Navie']=accnav[number]
        dataframe.loc[idex,'Decision']=accdes[number]
        dataframe.loc[idex,'Random']=accrf[number]
    return dataframe
    


In [42]:
result=pca_classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf)
result

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
PC-1,0.891892,0.891892,0.891892,0.891892,0.918919,0.702703,0.702703
PC-2,0.891892,0.891892,0.891892,0.891892,0.891892,0.783784,0.810811
PC-3,0.945946,0.945946,0.945946,0.945946,0.918919,0.810811,0.891892
PC-4,0.918919,0.891892,0.918919,0.891892,0.918919,0.756757,0.837838
PC-5,0.918919,0.891892,0.891892,0.891892,0.891892,0.783784,0.864865
